In [1]:
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Baseline

In [66]:
m = ConcreteModel()

# Define Sets
m.ITEMS = Set(initialize = ['A', 'B', 'C'])

# Decision Variable
m.MarkPct = Var(m.ITEMS, bounds = (0, 1))

# Define Params
m.basePrice = Param(m.ITEMS, initialize = {'A': 21.5, 'B' : 29.5, 'C': 30.5})
m.initInv = Param(m.ITEMS, initialize = {'A':10 , 'B':30 , 'C': 41})
m.maxPct = 0.8
m.cap = 15
m.dispCost = 1.2

def MarkPriceRule(m, i):
    return (1 - m.MarkPct[i])*m.basePrice[i]

def DemandRule(m, i):
    return m.MarkPct[i] * m.initInv[i]

def RevenueRule(m, i):
    return m.MarkPrice[i] * m.Demand[i] 

def FinalInvRule(m, i):
    return m.initInv[i] - m.Demand[i]

def MaxDiscountRule(m, i):
    return m.MarkPct[i] <= m.maxPct

def LeftoverInvRule(m, i):
    return m.FinalInv[i] <= m.cap

# Define implicit variables
m.MarkPrice = Expression(m.ITEMS, initialize = MarkPriceRule)
m.Demand = Expression(m.ITEMS, initialize = DemandRule)
m.Revenue = Expression(m.ITEMS, initialize = RevenueRule)
m.FinalInv = Expression(m.ITEMS, initialize = FinalInvRule)


m.MaxDiscount = Constraint(m.ITEMS, rule = MaxDiscountRule)
m.LeftOverInvRule = Constraint(m.ITEMS, rule = LeftoverInvRule)


def obj(m):
    return quicksum(m.Revenue[i] - m.dispCost * m.FinalInv[i] for i in m.ITEMS)

m.obj = Objective(rule = obj, sense = maximize)

In [67]:
from pyomo.contrib import appsi
SOLVER = appsi.solvers.Ipopt()

In [68]:
k = m.create_instance()
results = SOLVER.solve(k)

In [69]:
data = {'ITEM': [], 'Opt_Discount': [], 'Opt_Price': [], 'Opt_Revenue': [], 'Final_Inv': []}
for i in k.ITEMS:
    data['ITEM'].append(i)
    data['Opt_Discount'].append(value(k.MarkPct[i]))
    data['Opt_Price'].append(value(k.MarkPrice[i]))
    data['Opt_Revenue'].append(value(k.Revenue[i]))
    data['Final_Inv'].append(value(k.FinalInv[i]))
pd.DataFrame(data)

,ITEM,Opt_Discount,Opt_Price,Opt_Revenue,Final_Inv
0,A,0.527907,10.150000,53.582558,4.72093
1,B,0.520339,14.150000,220.883898,14.38983
2,C,0.634146,11.158537,290.121953,15.00000


In [70]:
print(k.obj())

523.655496557852


## Q1

In [71]:
m = ConcreteModel()

# Define Sets
m.ITEMS = Set(initialize = ['A', 'B', 'C'])

# Decision Variable
m.MarkPct = Var(m.ITEMS, bounds = (0, 1))

# Define Params
m.basePrice = Param(m.ITEMS, initialize = {'A': 21.5, 'B' : 29.5, 'C': 30.5})
m.initInv = Param(m.ITEMS, initialize = {'A':10 , 'B':30 , 'C': 41})
m.maxPct = 0.65
m.cap = 15
m.dispCost = 1.2

def MarkPriceRule(m, i):
    return (1 - m.MarkPct[i])*m.basePrice[i]

def DemandRule(m, i):
    return m.MarkPct[i] * m.initInv[i]

def RevenueRule(m, i):
    return m.MarkPrice[i] * m.Demand[i] 

def FinalInvRule(m, i):
    return m.initInv[i] - m.Demand[i]

def MaxDiscountRule(m, i):
    return m.MarkPct[i] <= m.maxPct

def LeftoverInvRule(m, i):
    return m.FinalInv[i] <= m.cap

# Define implicit variables
m.MarkPrice = Expression(m.ITEMS, initialize = MarkPriceRule)
m.Demand = Expression(m.ITEMS, initialize = DemandRule)
m.Revenue = Expression(m.ITEMS, initialize = RevenueRule)
m.FinalInv = Expression(m.ITEMS, initialize = FinalInvRule)


m.MaxDiscount = Constraint(m.ITEMS, rule = MaxDiscountRule)
m.LeftOverInvRule = Constraint(m.ITEMS, rule = LeftoverInvRule)


def obj(m):
    return quicksum(m.Revenue[i] - m.dispCost * m.FinalInv[i] for i in m.ITEMS)

m.obj = Objective(rule = obj, sense = maximize)

In [72]:
results = SOLVER.solve(m)

In [73]:
print(m.obj())

523.6554965577358


## Q2

In [74]:
m = ConcreteModel()

# Define Sets
m.ITEMS = Set(initialize = ['A', 'B', 'C'])

# Decision Variable
m.MarkPct = Var(m.ITEMS, bounds = (0, 1))

# Define Params
m.basePrice = Param(m.ITEMS, initialize = {'A': 21.5, 'B' : 29.5, 'C': 30.5})
m.initInv = Param(m.ITEMS, initialize = {'A':10 , 'B':30 , 'C': 41})
m.maxPct = 0.55
m.cap = 15
m.dispCost = 1.2

def MarkPriceRule(m, i):
    return (1 - m.MarkPct[i])*m.basePrice[i]

def DemandRule(m, i):
    return m.MarkPct[i] * m.initInv[i]

def RevenueRule(m, i):
    return m.MarkPrice[i] * m.Demand[i] 

def FinalInvRule(m, i):
    return m.initInv[i] - m.Demand[i]

def MaxDiscountRule(m, i):
    return m.MarkPct[i] <= m.maxPct

def LeftoverInvRule(m, i):
    return m.FinalInv[i] <= m.cap

# Define implicit variables
m.MarkPrice = Expression(m.ITEMS, initialize = MarkPriceRule)
m.Demand = Expression(m.ITEMS, initialize = DemandRule)
m.Revenue = Expression(m.ITEMS, initialize = RevenueRule)
m.FinalInv = Expression(m.ITEMS, initialize = FinalInvRule)


m.MaxDiscount = Constraint(m.ITEMS, rule = MaxDiscountRule)
m.LeftOverInvRule = Constraint(m.ITEMS, rule = LeftoverInvRule)


def obj(m):
    return quicksum(m.Revenue[i] - m.dispCost * m.FinalInv[i] for i in m.ITEMS)

m.obj = Objective(rule = obj, sense = maximize)

In [75]:
results = SOLVER.solve(m)

RuntimeError: A feasible solution was not found, so no solution can be loaded. If using the appsi.solvers.Ipopt interface, you can set opt.config.load_solution=False. If using the environ.SolverFactory interface, you can set opt.solve(model, load_solutions = False). Then you can check results.termination_condition and results.best_feasible_objective before loading a solution.

## Q4

In [76]:
m = ConcreteModel()

# Define Sets
m.ITEMS = Set(initialize = ['A', 'B', 'C'])

# Decision Variable
m.MarkPct = Var(m.ITEMS, bounds = (0, 1))

# Define Params
m.basePrice = Param(m.ITEMS, initialize = {'A': 21.5, 'B' : 29.5, 'C': 30.5})
m.initInv = Param(m.ITEMS, initialize = {'A':10 , 'B':30 , 'C': 41})
m.maxPct = 0.55
# m.cap = 15
m.dispCost = 1.2

def MarkPriceRule(m, i):
    return (1 - m.MarkPct[i])*m.basePrice[i]

def DemandRule(m, i):
    return m.MarkPct[i] * m.initInv[i]

def RevenueRule(m, i):
    return m.MarkPrice[i] * m.Demand[i] 

def FinalInvRule(m, i):
    return m.initInv[i] - m.Demand[i]

def MaxDiscountRule(m, i):
    return m.MarkPct[i] <= m.maxPct

# def LeftoverInvRule(m, i):
#     return m.FinalInv[i] <= m.cap

# Define implicit variables
m.MarkPrice = Expression(m.ITEMS, initialize = MarkPriceRule)
m.Demand = Expression(m.ITEMS, initialize = DemandRule)
m.Revenue = Expression(m.ITEMS, initialize = RevenueRule)
m.FinalInv = Expression(m.ITEMS, initialize = FinalInvRule)


m.MaxDiscount = Constraint(m.ITEMS, rule = MaxDiscountRule)
# m.LeftOverInvRule = Constraint(m.ITEMS, rule = LeftoverInvRule)


def obj(m):
    return quicksum(m.Revenue[i] - m.dispCost * m.FinalInv[i] for i in m.ITEMS)

m.obj = Objective(rule = obj, sense = maximize)

In [77]:
results = SOLVER.solve(m)

In [78]:
print(m.obj())

540.0424779816099


## Q4

In [88]:
m = ConcreteModel()

# Define Sets
m.ITEMS = Set(initialize = ['A', 'B', 'C'])

# Decision Variable
m.MarkPct = Var(m.ITEMS, bounds = (0, 1))

# Define Params
m.basePrice = Param(m.ITEMS, initialize = {'A': 21.5, 'B' : 29.5, 'C': 30.5})
m.initInv = Param(m.ITEMS, initialize = {'A':10 , 'B':30 , 'C': 41})
m.maxPct = 0.8
m.cap = 20
m.dispCost = 1.2

def MarkPriceRule(m, i):
    return (1 - m.MarkPct[i])*m.basePrice[i]

def DemandRule(m, i):
    return m.MarkPct[i] * m.initInv[i]

def RevenueRule(m, i):
    return m.MarkPrice[i] * m.Demand[i] 

def FinalInvRule(m, i):
    return m.initInv[i] - m.Demand[i]

def MaxDiscountRule(m, i):
    return m.MarkPct[i] <= m.maxPct

def LeftoverInvRule(m):
    return quicksum(m.FinalInv[i] for i in m.ITEMS) <= m.cap

# Define implicit variables
m.MarkPrice = Expression(m.ITEMS, initialize = MarkPriceRule)
m.Demand = Expression(m.ITEMS, initialize = DemandRule)
m.Revenue = Expression(m.ITEMS, initialize = RevenueRule)
m.FinalInv = Expression(m.ITEMS, initialize = FinalInvRule)


m.MaxDiscount = Constraint(m.ITEMS, rule = MaxDiscountRule)
m.LeftOverInvRule = Constraint(rule = LeftoverInvRule)


def obj(m):
    return quicksum(m.Revenue[i] - m.dispCost * m.FinalInv[i] for i in m.ITEMS)

m.obj = Objective(rule = obj, sense = maximize)

In [89]:
results = SOLVER.solve(m)

In [90]:
data = {'ITEM': [], 'Opt_Discount': [], 'Opt_Price': [], 'Opt_Revenue': [], 'Final_Inv': []}
for i in k.ITEMS:
    data['ITEM'].append(i)
    data['Opt_Discount'].append(round(value(m.MarkPct[i]) * 100, 2))
    data['Opt_Price'].append(round(value(m.MarkPrice[i]), 2))
    data['Opt_Revenue'].append(value(m.Revenue[i]))
    data['Final_Inv'].append(value(m.FinalInv[i]))
pd.DataFrame(data)

,ITEM,Opt_Discount,Opt_Price,Opt_Revenue,Final_Inv
0,A,80.00,4.30,34.399999,2.000000
1,B,75.12,7.34,165.389407,7.462933
2,C,74.30,7.84,238.785232,10.537068


## Q5

In [84]:
m = ConcreteModel()

# Define Sets
m.ITEMS = Set(initialize = ['A', 'B', 'C', 'D'])

# Decision Variable
m.MarkPct = Var(m.ITEMS, bounds = (0, 1))

# Define Params
m.basePrice = Param(m.ITEMS, initialize = {'A': 21.5, 'B' : 29.5, 'C': 30.5, 'D': 99})
m.initInv = Param(m.ITEMS, initialize = {'A':10 , 'B':30 , 'C': 41, 'D': 5})
m.maxPct = 0.8
m.cap = 15
m.dispCost = 1.2

def MarkPriceRule(m, i):
    return (1 - m.MarkPct[i])*m.basePrice[i]

def DemandRule(m, i):
    return m.MarkPct[i] * m.initInv[i]

def RevenueRule(m, i):
    return m.MarkPrice[i] * m.Demand[i] 

def FinalInvRule(m, i):
    return m.initInv[i] - m.Demand[i]

def MaxDiscountRule(m, i):
    return m.MarkPct[i] <= m.maxPct

def LeftoverInvRule(m, i):
    return m.FinalInv[i] <= m.cap

# Define implicit variables
m.MarkPrice = Expression(m.ITEMS, initialize = MarkPriceRule)
m.Demand = Expression(m.ITEMS, initialize = DemandRule)
m.Revenue = Expression(m.ITEMS, initialize = RevenueRule)
m.FinalInv = Expression(m.ITEMS, initialize = FinalInvRule)


m.MaxDiscount = Constraint(m.ITEMS, rule = MaxDiscountRule)
m.LeftOverInvRule = Constraint(m.ITEMS, rule = LeftoverInvRule)


def obj(m):
    return quicksum(m.Revenue[i] - m.dispCost * m.FinalInv[i] for i in m.ITEMS)

m.obj = Objective(rule = obj, sense = maximize)

In [85]:
results = SOLVER.solve(m)

In [86]:
print(round(m.obj(), 2))

644.42
